In [3]:
# Importing the necessary library
import boto3
import sagemaker
import pandas as pd
from joblib import dump, load
import s3fs


In [4]:
# Initialising new sagemaker session as "sess".
sess = sagemaker.Session()
# Bucket variable is used for storing the location of the bucket
bucket = 'sagemaker-studio-009676737623-l4vs7j0o0ib'
# Assigning the prefix variable 
prefix = 'mlops-level1-data' 
# Check for necessary permission needed for training and deploying models. 
role = sagemaker.get_execution_role()
# To understand where this session is configured to operate.
region = boto3.Session().region_name


## Create a Model Quality Baseline

In [5]:
from sagemaker import get_execution_role, session, Session
from sagemaker.model_monitor import ModelQualityMonitor , MonitoringDatasetFormat

model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=sess
)

In [12]:
baseline_dataset_uri = 's3://sagemaker-studio-009676737623-l4vs7j0o0ib/mlops-level1-data/data_drift_check_data/output/v1'
baseline_results_uri = 's3://sagemaker-studio-009676737623-l4vs7j0o0ib/mlops-level1-data/data_drift_check_data/result/v1'


In [15]:
df = pd.read_csv('s3://sagemaker-studio-009676737623-l4vs7j0o0ib/mlops-level1-data/data_drift_check_data/output/v1/v1_data.csv')

In [17]:
df


,"angle(X,gravityMean)",tGravityAcc-mean()-X,tGravityAcc-energy()-X,tGravityAcc-max()-X,tGravityAcc-min()-X,tGravityAcc-max()-Y,tBodyAcc-max()-X,tGravityAcc-mean()-Y,tGravityAcc-min()-Y,"angle(Y,gravityMean)",fBodyBodyAccJerkMag-entropy(),tGravityAcc-energy()-Y,label,prediction
0,-0.841247,0.963396,0.899469,0.892055,0.977436,-0.161265,-0.934724,-0.140840,-0.123213,0.179941,-1.000000,-0.970905,STANDING,STANDING
1,-0.844788,0.966561,0.907829,0.892060,0.984520,-0.161343,-0.943068,-0.141551,-0.114893,0.180289,-1.000000,-0.970583,STANDING,SITTING
2,-0.848933,0.966878,0.908668,0.892401,0.986770,-0.163711,-0.938692,-0.142010,-0.114893,0.180637,-1.000000,-0.970368,STANDING,STANDING
3,-0.848649,0.967615,0.910621,0.893817,0.986821,-0.163711,-0.938692,-0.143976,-0.121336,0.181935,-1.000000,-0.969400,STANDING,SITTING
4,-0.847865,0.968224,0.912235,0.893817,0.987434,-0.166786,-0.942469,-0.148750,-0.121834,0.185151,-1.000000,-0.967051,STANDING,SITTING
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,0.380157,-0.182499,-0.921584,-0.251484,-0.137947,0.801740,-0.930762,0.850167,0.863519,-0.644723,-1.000000,0.454273,LAYING,LAYING
605,0.380773,-0.183307,-0.922017,-0.251734,-0.139476,0.802381,-0.932471,0.851120,0.863519,-0.645972,-1.000000,0.457466,LAYING,LAYING
606,0.382072,-0.184870,-0.922850,-0.252703,-0.141864,0.802381,-0.938311,0.851608,0.864330,-0.645809,-1.000000,0.459104,LAYING,LAYING
607,0.383733,-0.186967,-0.923962,-0.254273,-0.143066,0.801912,-0.910459,0.850231,0.863769,-0.643164,-0.940965,0.454486,LAYING,LAYING


In [8]:
baseline_job_name = "model-quality-check-BaseLineJob"
job = model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri, # The S3 location of the validation dataset.
    dataset_format=MonitoringDatasetFormat.csv(header=True),
    output_s3_uri = baseline_results_uri, # The S3 location to store the results.
    problem_type='MulticlassClassification',
    inference_attribute= "prediction", # The column in the dataset that contains predictions.
    ground_truth_attribute= "label" # The column in the dataset that contains ground truth labels.
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name model-quality-check-BaseLineJob


..................................................................!

In [9]:
baseline_job = model_quality_monitor.latest_baselining_job

In [10]:
import pandas as pd
pd.DataFrame(baseline_job.suggested_constraints().body_dict["multiclass_classification_constraints"]).T

,threshold,comparison_operator
accuracy,0.880131,LessThanThreshold
weighted_recall,0.880131,LessThanThreshold
weighted_precision,0.884306,LessThanThreshold
weighted_f0_5,0.879333,LessThanThreshold
weighted_f1,0.876337,LessThanThreshold
weighted_f2,0.87748,LessThanThreshold


In [11]:
multiclass_metrics = baseline_job.baseline_statistics().body_dict["multiclasss_classification_metrics"]
pd.json_normalize(multiclass_metricss).T

KeyError: 'multiclasss_classification_metrics'